In [3]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("GPU devices:", tf.config.list_physical_devices("GPU"))


TensorFlow version: 2.20.0
GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# --- Dataset ---
img_size = (224, 224)
batch_size = 32
seed = 123

train_ds = tf.keras.utils.image_dataset_from_directory(
    "datasets",
    validation_split=0.2,
    subset="training",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "datasets",
    validation_split=0.2,
    subset="validation",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
print("Kelas:", class_names)

# --- Optimasi pipeline (prefetch) ---
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# --- Augmentasi ---
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])

# --- Base Model Transfer Learning ---
base_model = tf.keras.applications.MobileNetV2(
    input_shape=img_size + (3,),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False  # freeze dulu

# --- Model ---
model = keras.Sequential([
    data_augmentation,
    layers.Rescaling(1./255),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(len(class_names), activation="softmax")
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# --- Training ---
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

# --- Evaluasi ---
loss, acc = model.evaluate(val_ds)
print(f"\n✅ Akurasi validasi: {acc:.2f}")

# --- Simpan model ---
model.save("bicara_mobilenetv2.h5")


Found 4000 files belonging to 4 classes.
Using 3200 files for training.
Found 4000 files belonging to 4 classes.
Using 800 files for validation.
Kelas: ['C', 'I', 'L', 'V']
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 15s 93ms/step - accuracy: 0.4581 - loss: 1.2394 - val_accuracy: 0.8800 - val_loss: 0.6143
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 82ms/step - accuracy: 0.7816 - loss: 0.6310 - val_accuracy: 0.9850 - val_loss: 0.3169
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 82ms/step - accuracy: 0.9028 - loss: 0.3722 - val_accuracy: 0.9962 - val_loss: 0.1938
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 83ms/step - accuracy: 0.9531 - loss: 0.2543 - val_accuracy: 0.9975 - val_loss: 0.1251
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 83ms/step - accuracy: 0.9719 - loss: 0.1865 - val_accuracy: 0.9975 - val_loss: 0.0913
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 83ms/step - accuracy: 0.9819 - loss: 0.1460 - val_accuracy: 0.9975 - val_loss: 0.0696
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 84ms/step


✅ Akurasi validasi: 1.00


In [18]:
# 1. Import library
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pathlib
from tensorflow.keras.preprocessing import image

# 2. Path dataset dan model
dataset_dir = pathlib.Path('/home/antariksa/Desktop/Coding/bicara-model/datasets')
model_path = "bicara_mobilenetv2.h5"

# 3. Load model yang sudah dilatih
model = keras.models.load_model(model_path)
print("✅ Model berhasil dimuat!")

# 4. Ambil nama kelas dari folder dataset (sama dengan saat training)
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

class_names = train_ds.class_names
print("Kelas yang ditemukan:", class_names)

def predict_image(img_path):
    # Load gambar
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # JANGAN dibagi 255 (sudah ada Rescaling di model)

    # Prediksi (pastikan dropout nonaktif)
    predictions = model(img_array, training=False).numpy()
    score = tf.nn.softmax(predictions[0])

    predicted_class = class_names[np.argmax(score)]
    confidence = 100 * np.max(score)

    print(f"Gambar: {img_path}")
    print(f"Prediksi: {predicted_class} ({confidence:.2f}% confidence)")

    return predicted_class, confidence


# 6. Contoh penggunaan
img_path = "/home/antariksa/Desktop/Coding/bicara-model/datasets/V/Image_900.jpg"  # ganti sesuai gambar
predict_image(img_path)


✅ Model berhasil dimuat!
Found 4000 files belonging to 4 classes.
Using 3200 files for training.
Kelas yang ditemukan: ['C', 'I', 'L', 'V']
Gambar: /home/antariksa/Desktop/Coding/bicara-model/datasets/V/Image_900.jpg
Prediksi: V (43.88% confidence)


('V', 43.879154324531555)

In [14]:
import cv2
import time
import os
import numpy as np
import math
from cvzone.HandTrackingModule import HandDetector

# --- SETUP ---
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=2, detectionCon=0.7, minTrackCon=0.5)

offset = 25
imgSize = 300
folder = "datasets/B"
os.makedirs(folder, exist_ok=True)

# Variabel untuk cek stabil
stable_start_time = None
last_bbox = None
capture_delay = 2  # detik (Waktu tunggu awal sebelum mulai mengambil gambar)
max_images = 1000

# === LOGIKA BARU UNTUK PENGAMBILAN CEPAT ===
# Variabel untuk mengatur jeda antar pengambilan gambar setelah stabil
last_capture_time = 0
capture_interval = 0.1 # Ambil gambar setiap 0.1 detik jika tangan tetap stabil

# Definisi koneksi antar landmark jari (MediaPipe Hands style)
connections = [
    (0,1),(1,2),(2,3),(3,4),        # jempol
    (0,5),(5,6),(6,7),(7,8),        # telunjuk
    (5,9),(9,10),(10,11),(11,12),   # tengah
    (9,13),(13,14),(14,15),(15,16), # manis
    (13,17),(17,18),(18,19),(19,20),# kelingking
    (0,17)                          # telapak bawah
]

while True:
    success, img = cap.read()
    if not success:
        break

    img = cv2.flip(img, 1)
    hands, _ = detector.findHands(img, draw=False, flipType=False)

    if len(hands) == 2:
        # Ambil bounding box gabungan dari 2 tangan
        x1, y1, w1, h1 = hands[0]['bbox']
        x2, y2, w2, h2 = hands[1]['bbox']

        x_min = max(0, min(x1, x2) - offset)
        y_min = max(0, min(y1, y2) - offset)
        x_max = min(img.shape[1], max(x1 + w1, x2 + w2) + offset)
        y_max = min(img.shape[0], max(y1 + h1, y2 + h2) + offset)

        imgCrop = img[y_min:y_max, x_min:x_max].copy()

        if imgCrop.shape[0] > 0 and imgCrop.shape[1] > 0:
            # Gambar landmark di atas imgCrop
            for hand_data in hands:
                lmList = hand_data['lmList']

                # Titik merah
                for lx, ly, _ in lmList:
                    lx_crop, ly_crop = lx - x_min, ly - y_min
                    cv2.circle(imgCrop, (lx_crop, ly_crop), 5, (0, 0, 255), cv2.FILLED)

                # Garis putih
                for start, end in connections:
                    x1_lm, y1_lm, _ = lmList[start]
                    x2_lm, y2_lm, _ = lmList[end]
                    cv2.line(imgCrop, (x1_lm - x_min, y1_lm - y_min),
                                      (x2_lm - x_min, y2_lm - y_min), (255, 255, 255), 2)

            # === CEK STABILITAS DAN SIMPAN ===
            current_bbox = (x_min, y_min, x_max, y_max)
            if last_bbox is not None:
                diff = np.abs(np.array(current_bbox) - np.array(last_bbox)).sum()
                if diff < 30:  # artinya tangan relatif stabil
                    if stable_start_time is None:
                        stable_start_time = time.time()
                    else:
                        elapsed = time.time() - stable_start_time
                        cv2.putText(img, f"Stabil {elapsed:.1f}s", (50, 50),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                        # Tunggu hingga delay awal terpenuhi, lalu mulai pengambilan gambar cepat
                        if elapsed >= capture_delay:
                            
                            # === LOGIKA BARU DITERAPKAN DI SINI ===
                            # Cek apakah sudah waktunya mengambil gambar lagi berdasarkan interval
                            current_time = time.time()
                            if current_time - last_capture_time > capture_interval:
                                last_capture_time = current_time # Update waktu pengambilan terakhir
                                
                                count = len(os.listdir(folder))
                                if count >= max_images:
                                    print(f"Sudah mencapai batas maksimal gambar ({max_images}).")
                                    break

                                if imgCrop.size > 0:
                                    imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
                                    h_crop, w_crop, _ = imgCrop.shape
                                    aspectRatio = h_crop / w_crop
                                    if aspectRatio > 1:
                                        k = imgSize / h_crop
                                        wCal = math.ceil(k * w_crop)
                                        imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                                        wGap = math.ceil((imgSize - wCal) / 2)
                                        imgWhite[:, wGap:wCal + wGap] = imgResize
                                    else:
                                        k = imgSize / w_crop
                                        hCal = math.ceil(k * h_crop)
                                        imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                                        hGap = math.ceil((imgSize - hCal) / 2)
                                        imgWhite[hGap:hCal + hGap, :] = imgResize

                                    save_path = f"{folder}/Image_{count + 1}.jpg"
                                    cv2.imwrite(save_path, imgWhite)
                                    # Tambahkan print untuk melihat progres
                                    print(f"Gambar disimpan: {save_path} ({count + 1}/{max_images})")
                                    # stable_start_time TIDAK di-reset agar pengambilan berlanjut
                else:
                    # Reset timer jika tangan bergerak
                    stable_start_time = None
            last_bbox = current_bbox

            cv2.imshow("Gestur BISINDO (2 Tangan)", imgCrop)

    else:
        # Reset timer jika tangan tidak terdeteksi
        stable_start_time = None

    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1757736747.213396   95859 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1757736747.215667  142006 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.2), renderer: AMD Radeon Graphics (radeonsi, renoir, ACO, DRM 3.61, 6.14.0-29-generic)


Gambar disimpan: datasets/B/Image_1.jpg (1/1000)
Gambar disimpan: datasets/B/Image_2.jpg (2/1000)
Gambar disimpan: datasets/B/Image_3.jpg (3/1000)
Gambar disimpan: datasets/B/Image_4.jpg (4/1000)
Gambar disimpan: datasets/B/Image_5.jpg (5/1000)
Gambar disimpan: datasets/B/Image_6.jpg (6/1000)
Gambar disimpan: datasets/B/Image_7.jpg (7/1000)
Gambar disimpan: datasets/B/Image_8.jpg (8/1000)
Gambar disimpan: datasets/B/Image_9.jpg (9/1000)
Gambar disimpan: datasets/B/Image_10.jpg (10/1000)
Gambar disimpan: datasets/B/Image_11.jpg (11/1000)
Gambar disimpan: datasets/B/Image_12.jpg (12/1000)
Gambar disimpan: datasets/B/Image_13.jpg (13/1000)
Gambar disimpan: datasets/B/Image_14.jpg (14/1000)
Gambar disimpan: datasets/B/Image_15.jpg (15/1000)
Gambar disimpan: datasets/B/Image_16.jpg (16/1000)
Gambar disimpan: datasets/B/Image_17.jpg (17/1000)
Gambar disimpan: datasets/B/Image_18.jpg (18/1000)
Gambar disimpan: datasets/B/Image_19.jpg (19/1000)
Gambar disimpan: datasets/B/Image_20.jpg (20/1000